<a href="https://colab.research.google.com/github/navneetkrc/Colab_fastai/blob/master/Medicine_review_ULMFiT_fastai_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install Dependencies
Let's install PyTorch and fastai libraries first. You have to repeat this step every time you restart this notebook in colab.

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [2]:
import torch
print(torch.__version__)
import fastai
print(fastai.__version__) #1.0.38


1.0.0
1.0.38


In [0]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

## Import Dataset

###Install Kaggle Dependencies

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!mkdir -p input
!echo '{"username":"navneetkrch","key":"4910f3543e9486158ccfa00ca84b274f"}'> ~/.kaggle/kaggle.json  # Replace
#!chmod 600 kaggle.json


In [0]:
!kaggle datasets download -d jessicali9530/kuc-hackathon-winter-2018

In [0]:
!unzip kuc-hackathon-winter-2018.zip -d input


In [0]:
train = pd.read_csv('input/drugsComTrain_raw.csv')
test = pd.read_csv('input/drugsComTest_raw.csv')

In [8]:
train.head().T

,0,1,2,3,4
uniqueID,206461,95260,92703,138000,35696
drugName,Valsartan,Guanfacine,Lybrel,Ortho Evra,Buprenorphine / naloxone
condition,Left Ventricular Dysfunction,ADHD,Birth Control,Birth Control,Opiate Dependence
review,"""It has no side effect, I take it in combinati...","""My son is halfway through his fourth week of ...","""I used to take another oral contraceptive, wh...","""This is my first time using any form of birth...","""Suboxone has completely turned my life around..."
rating,9,8,5,8,9
date,20-May-12,27-Apr-10,14-Dec-09,3-Nov-15,27-Nov-16
usefulCount,27,192,17,10,37


In [9]:
test.head().T

,0,1,2,3,4
uniqueID,163740,206473,159672,39293,97768
drugName,Mirtazapine,Mesalamine,Bactrim,Contrave,Cyclafem 1 / 35
condition,Depression,"Crohn's Disease, Maintenance",Urinary Tract Infection,Weight Loss,Birth Control
review,"""I&#039;ve tried a few antidepressants over th...","""My son has Crohn&#039;s disease and has done ...","""Quick reduction of symptoms""","""Contrave combines drugs that were used for al...","""I have been on this birth control for one cyc..."
rating,10,8,9,9,9
date,28-Feb-12,17-May-09,29-Sep-17,5-Mar-17,22-Oct-15
usefulCount,22,17,3,35,4


In [10]:
train.shape

(161297, 7)

##Create Dataframe
This provides the Label/Target and the associated text

In [11]:
# create a dataframe
train_df = pd.DataFrame({'label':train.rating,
                   'text':train.review})
print(train_df.shape)
train_df.head()

(161297, 2)


,label,text
0,9,"""It has no side effect, I take it in combinati..."
1,8,"""My son is halfway through his fourth week of ..."
2,5,"""I used to take another oral contraceptive, wh..."
3,8,"""This is my first time using any form of birth..."
4,9,"""Suboxone has completely turned my life around..."


In [12]:
test_df = pd.DataFrame({'label':test.rating,
                   'text':test.review})
print(test_df.shape)
test_df.head()

(53766, 2)


,label,text
0,10,"""I&#039;ve tried a few antidepressants over th..."
1,8,"""My son has Crohn&#039;s disease and has done ..."
2,9,"""Quick reduction of symptoms"""
3,9,"""Contrave combines drugs that were used for al..."
4,9,"""I have been on this birth control for one cyc..."


In [0]:
df= train_df.append(test_df, ignore_index=True)


In [14]:
df.shape

(215063, 2)

In [15]:
#df[df['label'].isin([1,10])]
df = df.reset_index(drop = True)
df.head()

,label,text
0,9,"""It has no side effect, I take it in combinati..."
1,8,"""My son is halfway through his fourth week of ..."
2,5,"""I used to take another oral contraceptive, wh..."
3,8,"""This is my first time using any form of birth..."
4,9,"""Suboxone has completely turned my life around..."


In [16]:
df['label'].value_counts()

10    68005
9     36708
1     28918
8     25046
7     12547
5     10723
2      9265
3      8718
6      8462
4      6671
Name: label, dtype: int64

## Preprocessing

Tokenization

Remove StopWords

In [0]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english') 

In [0]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) 

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
df['text'] = detokenized_doc

In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.3, random_state = 12)

In [22]:
df_trn.shape, df_val.shape

((150544, 2), (64519, 2))

## Data Preparation

### Language model data
for learning the Language semantics

### Classifier model data
for learning the classification Task on text data

**text.learner contains helper functions to quickly create a language model or an RNN classifier**


In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

In [0]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

This does all the necessary preprocessing behing the scene. For the classifier, we also pass the vocabulary (mapping from ids to words) that we want to use: this is to ensure that data_clas will use the same dictionary as data_lm.



In [0]:
#Fine-tuning a language model
#We can use the data_lm object we created earlier to fine-tune a pretrained language model.
#fast.ai has an English model available that we can download. 
#We can create a learner object that will directly create a model, 
#download the pretrained weights and be ready for fine-tuning.
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [0]:
# train the learner object
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy


In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.save_encoder('ft_enc')
#Finally we save the encoder to be able to use it for classification in the next section.

##Building a classifier
We now use the data_clas object we created earlier to build a classifier with our fine-tuned encoder. The learner object can be done in a single line.



In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(1, 1e-2)